<a href="https://colab.research.google.com/github/drbronson/Auto-GPT/blob/master/Tableau_Server_Manual_Domain_Migration_Scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =====================================================================================================================
# General Setup and Prerequisites:
# ---------------------------------------------------------------------------------------------------------------------
# These scripts use Python 3.x and require the 'requests' library for REST API interactions.
# Install with: pip install requests
#
# For tabcmd operations, ensure tabcmd is installed and configured on the machine running these scripts.
# Reference: https://help.tableau.com/current/server/en-us/tabcmd.htm
#
# IMPORTANT: Replace placeholder values (e.g., "your_tableau_server_url", "admin_username", "admin_password",
#            "new_eads_domain", "old_domain_nickname", "eads_domain_nickname") with your actual environment details.
#
# These scripts are illustrative and may need adjustments based on your specific Tableau Server version,
# environment configuration, and data structures.
# Always test thoroughly in a non-production environment first!
# =====================================================================================================================

import requests
import json
import csv
import subprocess
import time
import sys

# =====================================================================================================================
# Configuration Variables (Update these for your environment)
# =====================================================================================================================

# Tableau Server Details (for the NEW EADS server for API calls)
TABLEAU_SERVER_URL = "https://your_tableau_server_url"  # e.g., "https://tableau.yourcompany.com"
TABLEAU_SERVER_VERSION = "3.22"  # API version, e.g., "3.22" for 2023.1.x, "3.23" for 2024.1.x, adjust as needed.

# Authentication for Tableau REST API (RECOMMENDED: Personal Access Token)
PAT_NAME = "MigrationScriptPAT"
PAT_SECRET = "your_personal_access_token_secret" # Generate this on Tableau Server
# OR (Less Secure for scripts):
# ADMIN_USERNAME = "your_admin_username"
# ADMIN_PASSWORD = "your_admin_password"

# File Paths for CSVs
USER_MAPPING_CSV = "user_mapping.csv"               # Input for ownership/permissions
NEW_USERS_CSV = "new_eads_users_to_add.csv"         # Input for bulk user creation
PERMISSION_MAPPING_CSV = "permission_mapping.csv"   # Input for bulk permission re-application

# =====================================================================================================================
# Helper Function: Tableau REST API Authentication
# =====================================================================================================================

def get_tableau_auth_token():
    """
    Authenticates to Tableau Server using Personal Access Token and returns the auth token.
    Alternatively, can be modified to use username/password.
    """
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    signin_url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/auth/signin"

    # Using Personal Access Token (Recommended)
    payload = {
        "credentials": {
            "personalAccessTokenName": PAT_NAME,
            "personalAccessTokenSecret": PAT_SECRET,
            "site": {
                "contentUrl": "" # Default site. Specify if signing into a non-default site.
            }
        }
    }

    # Using Username/Password (Less Secure for scripts)
    # payload = {
    #     "credentials": {
    #         "name": ADMIN_USERNAME,
    #         "password": ADMIN_PASSWORD,
    #         "site": {
    #             "contentUrl": ""
    #         }
    #     }
    # }

    try:
        response = requests.post(signin_url, json=payload, headers=headers)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        response_json = response.json()
        token = response_json['credentials']['token']
        site_id = response_json['credentials']['site']['id']
        user_id = response_json['credentials']['user']['id']
        print("Successfully authenticated to Tableau Server.")
        return token, site_id, user_id
    except requests.exceptions.RequestException as e:
        print(f"Authentication failed: {e}")
        sys.exit(1) # Exit if authentication fails

# =====================================================================================================================
# Script 1: tabcmd Bulk User Creation (Python using subprocess)
#
# Input CSV Format (new_eads_users_to_add.csv):
# username,password,site_name,role
# eads_domain\john.doe,P@ssw0rd1,Default,Explorer
# eads_domain\jane.smith,P@ssw0rd2,Sales,Viewer
#
# NOTE: Passwords are required by tabcmd createusers when not using --no-password.
# If your EADS setup involves SSO (e.g., SAML), passwords might not be strictly
# necessary for initial creation if users will sign in via SSO.
# Consider using --no-password if appropriate for your setup, but be aware of implications.
# =====================================================================================================================

def create_users_from_csv(csv_filepath):
    """
    Reads user data from a CSV and uses tabcmd to create users on Tableau Server.
    Assumes tabcmd is already logged into the target (new EADS) Tableau Server.
    """
    print(f"\n--- Starting bulk user creation from {csv_filepath} ---")
    try:
        with open(csv_filepath, mode='r', newline='') as file:
            reader = csv.DictReader(file)
            for row in reader:
                username = row.get('username')
                password = row.get('password')
                site_name = row.get('site_name')
                role = row.get('role')

                if not all([username, site_name, role]):
                    print(f"Skipping row due to missing data: {row}")
                    continue

                # Construct the tabcmd command
                # Adjust if your setup uses --no-password for SSO.
                command = [
                    "tabcmd", "createusers",
                    "--file", "-", # Read from stdin
                    "--site", site_name,
                    "--role", role
                ]

                # Run tabcmd using subprocess, passing CSV data via stdin
                # It's safer to pass credentials via a secure method if possible,
                # but for --file - tabcmd expects CSV format.
                csv_line = f"{username},{password},{role}\n" # tabcmd createusers --file - expects username,password,role

                try:
                    process = subprocess.run(command, input=csv_line.encode('utf-8'),
                                             capture_output=True, text=True, check=True)
                    print(f"Successfully created/updated user {username} on site {site_name}.")
                    # print(process.stdout) # Uncomment for detailed tabcmd output
                except subprocess.CalledProcessError as e:
                    print(f"Error creating user {username} on site {site_name}: {e.stderr}")
                except Exception as e:
                    print(f"An unexpected error occurred for user {username}: {e}")
        print("--- Bulk user creation complete ---")
    except FileNotFoundError:
        print(f"Error: CSV file not found at {csv_filepath}")
    except Exception as e:
        print(f"An error occurred during CSV reading: {e}")

# =====================================================================================================================
# Script 2: Tableau REST API Bulk Content Ownership Transfer
#
# Input CSV Format (user_mapping.csv):
# content_type,content_name,site_name,old_username,new_username
# Workbook,Sales Dashboard,Default,iApp\john.doe,eads_domain\john.doe
# Datasource,Superstore Data,Default,iApp\jane.smith,eads_domain\jane.smith
# Project,Marketing Analytics,Sales,iApp\david.lee,eads_domain\david.lee
#
# Supported content_type: "Workbook", "Datasource", "Project", "Flow" (Flows are more complex)
# =====================================================================================================================

def transfer_content_ownership(csv_filepath):
    """
    Transfers ownership of Tableau content (workbooks, datasources, projects) based on a CSV mapping.
    Uses Tableau Server REST API.
    """
    print(f"\n--- Starting content ownership transfer from {csv_filepath} ---")
    auth_token, site_id, current_user_id = get_tableau_auth_token()
    headers = {
        'X-Tableau-Auth': auth_token,
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    def get_object_id(object_type, object_name, site_content_url):
        """Helper to get ID of workbook, datasource, project, or flow."""
        url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/sites/{site_id}/{object_type.lower()}s?filter=name:eq:{object_name}"
        if site_content_url: # Only apply filter if not default site
            url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/sites/{site_id}/{object_type.lower()}s?filter=name:eq:{object_name}&site.contentUrl:eq:{site_content_url}"

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            items = response.json().get(f'{object_type.lower()}s', {}).get(object_type.lower(), [])
            if items:
                return items[0]['id']
            return None
        except requests.exceptions.RequestException as e:
            print(f"Error getting ID for {object_type} '{object_name}': {e}")
            return None

    def get_user_id_by_name(username):
        """Helper to get user ID by username."""
        url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/sites/{site_id}/users?filter=name:eq:{username}"
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            users = response.json().get('users', {}).get('user', [])
            if users:
                return users[0]['id']
            return None
        except requests.exceptions.RequestException as e:
            print(f"Error getting user ID for '{username}': {e}")
            return None

    try:
        with open(csv_filepath, mode='r', newline='') as file:
            reader = csv.DictReader(file)
            for row in reader:
                content_type = row.get('content_type')
                content_name = row.get('content_name')
                site_name = row.get('site_name', '') # Default site is empty string
                old_username = row.get('old_username')
                new_username = row.get('new_username')

                if not all([content_type, content_name, new_username]):
                    print(f"Skipping row due to missing data: {row}")
                    continue

                print(f"Processing {content_type} '{content_name}' on site '{site_name}' from '{old_username}' to '{new_username}'...")

                # Get Content ID
                content_id = get_object_id(content_type, content_name, site_name)
                if not content_id:
                    print(f"  {content_type} '{content_name}' not found. Skipping.")
                    continue

                # Get New Owner User ID
                new_owner_id = get_user_id_by_name(new_username)
                if not new_owner_id:
                    print(f"  New owner '{new_username}' not found on Tableau Server. Skipping.")
                    continue

                # Transfer Ownership
                if content_type.lower() == "workbook":
                    transfer_url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/sites/{site_id}/workbooks/{content_id}"
                elif content_type.lower() == "datasource":
                    transfer_url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/sites/{site_id}/datasources/{content_id}"
                elif content_type.lower() == "project":
                    transfer_url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/sites/{site_id}/projects/{content_id}"
                elif content_type.lower() == "flow":
                    transfer_url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/sites/{site_id}/flows/{content_id}"
                else:
                    print(f"  Unsupported content type '{content_type}'. Skipping.")
                    continue

                payload = {
                    "workbook" if content_type.lower() == "workbook" else
                    "datasource" if content_type.lower() == "datasource" else
                    "project" if content_type.lower() == "project" else
                    "flow" if content_type.lower() == "flow" else "": {
                        "owner": {
                            "id": new_owner_id
                        }
                    }
                }

                try:
                    response = requests.put(transfer_url, json=payload, headers=headers)
                    response.raise_for_status()
                    print(f"  Successfully transferred ownership of {content_type} '{content_name}' to '{new_username}'.")
                except requests.exceptions.RequestException as e:
                    print(f"  Error transferring ownership for {content_type} '{content_name}': {e}")

                time.sleep(0.1) # Small delay to avoid overwhelming the API
        print("--- Content ownership transfer complete ---")
    except FileNotFoundError:
        print(f"Error: CSV file not found at {csv_filepath}")
    except Exception as e:
        print(f"An error occurred during CSV reading or processing: {e}")

# =====================================================================================================================
# Script 3: Tableau REST API Bulk Permission Re-application
#
# Input CSV Format (permission_mapping.csv):
# content_type,content_name,site_name,group_name_or_username,capability,mode,grantee_type
# Project,Sales Reporting,Default,EADS_Sales_Team,Read,Allow,Group
# Workbook,Executive Summary,Default,eads_domain\jane.smith,Read,Allow,User
# Datasource,Customer Data,Default,EADS_Data_Stewards,Write,Allow,Group
#
# Supported content_type: "Workbook", "Datasource", "Project", "Flow"
# Supported capability: Read, Write, Delete, ChangePermissions, WebAuthoring, ExportData, Share etc.
#                       Refer to Tableau REST API documentation for full list per content type.
# Supported mode: "Allow", "Deny"
# Supported grantee_type: "Group", "User"
# =====================================================================================================================

def reapply_permissions(csv_filepath):
    """
    Re-applies permissions to Tableau content (workbooks, datasources, projects) based on a CSV mapping.
    Uses Tableau Server REST API.
    """
    print(f"\n--- Starting permission re-application from {csv_filepath} ---")
    auth_token, site_id, current_user_id = get_tableau_auth_token()
    headers = {
        'X-Tableau-Auth': auth_token,
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    def get_object_id(object_type, object_name, site_content_url):
        """Helper to get ID of workbook, datasource, project, or flow."""
        url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/sites/{site_id}/{object_type.lower()}s?filter=name:eq:{object_name}"
        if site_content_url:
            url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/sites/{site_id}/{object_type.lower()}s?filter=name:eq:{object_name}&site.contentUrl:eq:{site_content_url}"
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            items = response.json().get(f'{object_type.lower()}s', {}).get(object_type.lower(), [])
            if items:
                return items[0]['id']
            return None
        except requests.exceptions.RequestException as e:
            print(f"Error getting ID for {object_type} '{object_name}': {e}")
            return None

    def get_grantee_id(grantee_type, grantee_name):
        """Helper to get Group or User ID."""
        if grantee_type.lower() == 'group':
            url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/sites/{site_id}/groups?filter=name:eq:{grantee_name}"
            try:
                response = requests.get(url, headers=headers)
                response.raise_for_status()
                groups = response.json().get('groups', {}).get('group', [])
                if groups:
                    return groups[0]['id']
                return None
            except requests.exceptions.RequestException as e:
                print(f"Error getting Group ID for '{grantee_name}': {e}")
                return None
        elif grantee_type.lower() == 'user':
            url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/sites/{site_id}/users?filter=name:eq:{grantee_name}"
            try:
                response = requests.get(url, headers=headers)
                response.raise_for_status()
                users = response.json().get('users', {}).get('user', [])
                if users:
                    return users[0]['id']
                return None
            except requests.exceptions.RequestException as e:
                print(f"Error getting User ID for '{grantee_name}': {e}")
                return None
        return None

    try:
        with open(csv_filepath, mode='r', newline='') as file:
            reader = csv.DictReader(file)
            for row in reader:
                content_type = row.get('content_type')
                content_name = row.get('content_name')
                site_name = row.get('site_name', '')
                grantee_name = row.get('group_name_or_username')
                capability = row.get('capability')
                mode = row.get('mode')
                grantee_type = row.get('grantee_type')

                if not all([content_type, content_name, grantee_name, capability, mode, grantee_type]):
                    print(f"Skipping row due to missing data: {row}")
                    continue

                print(f"Processing permission for {content_type} '{content_name}' on site '{site_name}' for {grantee_type} '{grantee_name}'...")

                # Get Content ID
                content_id = get_object_id(content_type, content_name, site_name)
                if not content_id:
                    print(f"  {content_type} '{content_name}' not found. Skipping permission.")
                    continue

                # Get Grantee (User/Group) ID
                grantee_id = get_grantee_id(grantee_type, grantee_name)
                if not grantee_id:
                    print(f"  {grantee_type} '{grantee_name}' not found on Tableau Server. Skipping permission.")
                    continue

                # Add Permission
                if content_type.lower() == "workbook":
                    permission_url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/sites/{site_id}/workbooks/{content_id}/permissions"
                elif content_type.lower() == "datasource":
                    permission_url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/sites/{site_id}/datasources/{content_id}/permissions"
                elif content_type.lower() == "project":
                    permission_url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/sites/{site_id}/projects/{content_id}/permissions"
                elif content_type.lower() == "flow":
                    permission_url = f"{TABLEAU_SERVER_URL}/api/{TABLEAU_SERVER_VERSION}/sites/{site_id}/flows/{content_id}/permissions"
                else:
                    print(f"  Unsupported content type '{content_type}'. Skipping permission.")
                    continue

                # Construct payload based on grantee type
                payload = {
                    "permissions": {
                        "granteeCapabilities": {
                            "capability": [
                                {
                                    "name": capability,
                                    "mode": mode
                                }
                            ]
                        }
                    }
                }
                if grantee_type.lower() == 'group':
                    payload["permissions"]["group"] = {"id": grantee_id}
                else: # User
                    payload["permissions"]["user"] = {"id": grantee_id}


                try:
                    response = requests.put(permission_url, json=payload, headers=headers)
                    response.raise_for_status()
                    print(f"  Successfully applied {capability} {mode} permission to {content_type} '{content_name}' for {grantee_type} '{grantee_name}'.")
                except requests.exceptions.RequestException as e:
                    print(f"  Error applying permission for {content_type} '{content_name}' for {grantee_type} '{grantee_name}': {e}")

                time.sleep(0.1) # Small delay
        print("--- Permission re-application complete ---")
    except FileNotFoundError:
        print(f"Error: CSV file not found at {csv_filepath}")
    except Exception as e:
        print(f"An error occurred during CSV reading or processing: {e}")


# =====================================================================================================================
# Main execution block
# =====================================================================================================================

if __name__ == "__main__":
    # --- Phase 2.1: Whitelist Old Domain on New Server (TSM command) ---
    print("\n--- Phase 2.1: Whitelist Old Domain on New Server ---")
    print("Run the following TSM commands on your NEW EADS Tableau Server:")
    print(f"tsm configuration set -k wgserver.domain.accept_list -v \"old_domain_nickname,eads_domain_nickname\" --force-keys")
    print("tsm pending-changes apply")
    print("Remember to replace 'old_domain_nickname' and 'eads_domain_nickname' with your actual values.")
    print("This step temporarily allows the new server to recognize users from both domains during migration.")

    # --- Phase 2.2: Add EADS Users (and groups) to New Tableau Server ---
    # Make sure your new_eads_users_to_add.csv is ready.
    # Before running, ensure tabcmd is logged in to the new Tableau Server:
    # tabcmd login -s https://your_tableau_server_url -u admin_username -p admin_password --site Default
    # (Adjust site name as needed)
    print("\n--- Phase 2.2: Add EADS Users to New Tableau Server ---")
    print(f"Ensure tabcmd is logged in to {TABLEAU_SERVER_URL}")
    create_users_from_csv(NEW_USERS_CSV)

    # --- Phase 2.3 & 2.4: Export Content from Old Server & Import to New Server ---
    print("\n--- Phase 2.3 & 2.4: Content Export from Old Server & Import to New ---")
    print("These steps require iterating through your old Tableau Server content and then publishing to the new one.")
    print("This is typically done via shell/batch scripts or manually. Examples below:")
    print("\n# Example Shell/Batch Script for Exporting Workbooks (from OLD Tableau Server)")
    print("# First, log tabcmd into the OLD Tableau Server.")
    print("tabcmd login -s https://old_tableau_server_url -u old_admin_username -p old_admin_password --site Default")
    print("for site in Default Site1 Site2; do")
    print("  mkdir -p ./exports/$site")
    print("  tabcmd export --site $site --fullpdf -t all -f \"./exports/$site/workbooks.pdf\" # For PDF backup")
    print("  tabcmd export -t all --workbook \"*\" -f \"./exports/$site/workbooks\" # For .twb/.twbx files (loop projects if needed)")
    print("  # To export each workbook individually (better for specific re-publishing):")
    print("  # tabcmd listworkbooks --site $site > workbooks_list_$site.txt")
    print("  # while IFS= read -r workbook_name; do")
    print("  #   tabcmd get \"workbooks/$workbook_name\" --site $site -f \"./exports/$site/$workbook_name.twbx\"")
    print("  # done < workbooks_list_$site.txt")
    print("done")
    print("\n# Example Shell/Batch Script for Exporting Published Data Sources (from OLD Tableau Server)")
    print("tabcmd login -s https://old_tableau_server_url -u old_admin_username -p old_admin_password --site Default")
    print("for site in Default Site1 Site2; do")
    print("  mkdir -p ./exports/$site/datasources")
    print("  tabcmd listdatasources --site $site > datasources_list_$site.txt")
    print("  # while IFS= read -r datasource_name; do")
    print("  #   tabcmd get \"datasources/$datasource_name.tds\" --site $site -f \"./exports/$site/datasources/$datasource_name.tds\"")
    print("  # done < datasources_list_$site.txt")
    print("done")

    print("\n# Example Shell/Batch Script for Publishing Exported Content (to NEW EADS Tableau Server)")
    print("# First, log tabcmd into the NEW EADS Tableau Server.")
    print(f"tabcmd login -s {TABLEAU_SERVER_URL} -u admin_username -p admin_password --site Default")
    print("for site in Default Site1 Site2; do")
    print("  for workbook_file in ./exports/$site/*.twbx; do")
    print("    tabcmd publish \"$workbook_file\" --site $site --replace --no-cookie # --no-cookie can help with batch")
    print("  done")
    print("  for datasource_file in ./exports/$site/datasources/*.tds; do")
    print("    tabcmd publish \"$datasource_file\" --site $site --replace --no-cookie")
    print("  done")
    print("done")
    print("Remember to replace site names, and adjust paths as needed.")

    # --- Phase 2.5: Transfer Content Ownership ---
    # Make sure your user_mapping.csv is ready.
    print("\n--- Phase 2.5: Transfer Content Ownership (Python Script) ---")
    transfer_content_ownership(USER_MAPPING_CSV)

    # --- Phase 2.6: Re-apply Permissions ---
    # Make sure your permission_mapping.csv is ready.
    print("\n--- Phase 2.6: Re-apply Permissions (Python Script) ---")
    reapply_permissions(PERMISSION_MAPPING_CSV)

    # --- Phase 3.4: Final LDAP Configuration (TSM command) ---
    print("\n--- Phase 3.4: Final LDAP Configuration (on NEW EADS Server) ---")
    print("After all content and permissions are verified, run these TSM commands:")
    print(f"tsm configuration set -k wgserver.domain.accept_list -v \"eads_domain_nickname\" --force-keys")
    print("tsm pending-changes apply")
    print("This removes the old domain from the whitelist, enforcing EADS as the sole identity provider.")

    print("\n--- All planned scripts and commands outlined. ---")
    print("Remember to perform thorough validation and testing at each stage.")